# Analiza jakości i czystości danych dla każdego pliku w 202301

Jakość badana będzie pod względem:
- Ilości danych
- Różnorodności danych
- Kompletności 
- Maksymalnych i minimalnych wartości danych (w celach optymalizacyjnych)

## 1. Ilość danych

Do tego zadania wykorzystam jedno z dostępnych poleceń systemu linux  ``` wc -l *```  w folderze z plikami, w celu otrzymania informacji na temat ilości rekordów dostępnych w każdym pliku, wyniki prezentują się następująco:

| Lines     | File Name            |
|-----------|----------------------|
| 2776854   | curr_datausage.csv   |
| 1380352   | curr_dlping.csv      |
| 129561205 | curr_dns.csv         |
| 1         | curr_httpget.csv     |
| 1         | curr_httpgetmt6.csv  |
| 806143    | curr_httpgetmt.csv   |
| 1         | curr_httppost.csv    |
| 3         | curr_httppostmt6.csv |
| 803437    | curr_httppostmt.csv  |
| 977726    | curr_lct_dl.csv      |
| 975855    | curr_lct_ul.csv      |
| 4482226   | curr_ping.csv        |
| 13265008  | curr_traceroute.csv  |
| 2138360   | curr_udpcloss.csv    |
| 2688578   | curr_udpjitter.csv   |
| 18544     | curr_udplatency6.csv |
| 5454687   | curr_udplatency.csv  |
| 1394991   | curr_ulping.csv      |
| 10668403  | curr_webget.csv      |

Korzystając z powyższych danych już możemy określić, że że pliki **curr_httpget.csv**, **curr_httpgetmt6.csv**, **curr_httppost.csv** oraz **curr_httppostmt6** nie są warte analizy ze względu na bardzo małą ilość rekordów bądz ich brak (jedna linijka zawsze jest headerem csv), zatem przechodząc do kolejnych kroków nie zamierzam już analizować powyższych plików

## 2. Różnorodność danych dla poszczególnych kolumn w tabelach

W tym punkcie planuje skupić się na różnorodności danych oraz przedziałach wartości (oczywiście w zależności od typów przechowywanych typów) w celu analizy pod kątem kompletności, faktycznej różnorodności, oraz w celach optymalizacyjnych dla późniejszych problemów analitycznych

In [ ]:
import pandas as pd
import os

def analyze_large_dataframe(file_path, chunksize=100_000):
    analysis = {}

    for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=True):
        for column in chunk.columns:
            col_data = chunk[column]

            if pd.api.types.is_numeric_dtype(col_data):
                if column not in analysis:
                    analysis[column] = {
                        "min": col_data.min(),
                        "max": col_data.max(),
                        "null_values": col_data.isnull().sum()
                    }
                else:
                    analysis[column]["min"] = min(analysis[column]["min"], col_data.min())
                    analysis[column]["max"] = max(analysis[column]["max"], col_data.max())
                    analysis[column]["null_values"] += col_data.isnull().sum()

            elif pd.api.types.is_string_dtype(col_data):
                if column not in analysis:
                    analysis[column] = {
                        "max_length": col_data.dropna().map(len).max(),
                        "null_values": col_data.isnull().sum()
                    }
                else:
                    analysis[column]["max_length"] = max(analysis[column]["max_length"], col_data.dropna().map(len).max())
                    analysis[column]["null_values"] += col_data.isnull().sum()

            else:
                if column not in analysis:
                    analysis[column] = {
                        "type": "Non-numeric/non-string column",
                        "null_values": col_data.isnull().sum()
                    }
                else:
                    analysis[column]["null_values"] += col_data.isnull().sum()

    return analysis

def analyze_files_in_directory(directory_path):
    results = {}
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)

        if os.path.isfile(file_path) and file_name.endswith('.csv'):
            try:
                print(f"Processing: {file_name}")
                results[file_name] = analyze_large_dataframe(file_path)
            except Exception as e:
                results[file_name] = f"Error processing file: {e}"
    return results


A teraz użycie oraz wyniki:

In [13]:
analyze_files_in_directory("/home/arut3n/Documents/Studia/MagSem1/PAMSI/PAMSI-part-one/data/202301")

Processing: curr_httppostmt.csv
Processing: curr_httpgetmt.csv
Processing: curr_udpjitter.csv
Processing: curr_ulping.csv
Processing: curr_udplatency.csv
Processing: curr_webget.csv


/tmp/ipykernel_4333/1123069309.py:7: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=True):


Processing: curr_dns.csv


/tmp/ipykernel_4333/1123069309.py:7: DtypeWarning: Columns (0,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=True):
/tmp/ipykernel_4333/1123069309.py:7: DtypeWarning: Columns (0,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=True):
/tmp/ipykernel_4333/1123069309.py:7: DtypeWarning: Columns (0,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=True):
/tmp/ipykernel_4333/1123069309.py:7: DtypeWarning: Columns (0,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=True):
/tmp/ipykernel_4333/1123069309.py:7: DtypeWarning: Columns (0,5,6,7) have mixed types. Specify dtype option on import or

Processing: curr_datausage.csv
Processing: curr_httpgetmt6.csv
Processing: curr_udplatency6.csv
Processing: curr_ping.csv
Processing: curr_udpcloss.csv
Processing: curr_dlping.csv
Processing: curr_lct_ul.csv
Processing: curr_traceroute.csv
Processing: curr_lct_dl.csv
Processing: curr_httppostmt6.csv
Processing: curr_httpget.csv
Processing: curr_httppost.csv


{'curr_httppostmt.csv': {'unit_id': {'min': np.int64(386),
   'max': np.int64(90021129),
   'null_values': np.int64(0)},
  'dtime': {'max_length': np.int64(19), 'null_values': np.int64(0)},
  'target': {'max_length': np.int64(48), 'null_values': np.int64(0)},
  'address': {'max_length': np.int64(15), 'null_values': np.int64(0)},
  'fetch_time': {'min': np.int64(0),
   'max': np.int64(13071461),
   'null_values': np.int64(0)},
  'bytes_total': {'min': np.int64(0),
   'max': np.int64(1191124800),
   'null_values': np.int64(0)},
  'bytes_sec': {'min': np.int64(0),
   'max': np.int64(125220794),
   'null_values': np.int64(0)},
  'bytes_sec_interval': {'min': np.int64(0),
   'max': np.int64(125220794),
   'null_values': np.int64(0)},
  'warmup_time': {'min': np.int64(-2129616921),
   'max': np.int64(2140849142),
   'null_values': np.int64(0)},
  'warmup_bytes': {'min': np.int64(0),
   'max': np.int64(598577928),
   'null_values': np.int64(0)},
  'sequence': {'min': np.int64(0),
   'max': np